# Lightmap Visualizer

In [4]:
from pathlib import Path
import os

from data_structures.scene import Scene
from data_structures.color import Color

from renderer import Renderer
from lightmapper import Lightmapper

In [5]:
# --- USER CONFIGURATION REQUIRED ---          
scene_filenames = 'Milestone4'         # choose the scene and the lightmap to be loaded
viewport_width = 1920                       # width of the viewport in pixels
viewport_height = 1080                      # height of the viewport in pixels
atmosphere_color = Color(0.01, 0.01, 0.01)     # Color of the atmosphere, has no effect on lighting
# --- USER CONFIGURATION REQUIRED ---

#assets_path = Path('/home/devbox/Bayernstein/assets')
assets_path = Path('D:/data/Informatik/GamesEngineering/Bayernstein/assets')
#assets_path = Path('/Users/fabiandepaoli/Library/Mobile Documents/com~apple~CloudDocs/SharedData/HM/GamesEngineering/Bayernstein/assets')
#compiled_path = Path('/home/devbox/Bayernstein/assets/compiled')
compiled_path = Path('D:/data/Informatik/GamesEngineering/Bayernstein/assets/compiled')
#compiled_path = Path('/Users/fabiandepaoli/Library/Mobile Documents/com~apple~CloudDocs/SharedData/HM/GamesEngineering/Bayernstein/assets/compiled')
scene_path = compiled_path / scene_filenames / f'{scene_filenames}.json'
lightmap_path = compiled_path / scene_filenames / f'{scene_filenames}.hdr'
scene = Scene(scene_path, assets_path, lightmap_path)
#scene.create_frames(patch_resolution=0.0625)

scene.generate_vertex_array()
#scene.generate_line_array()

#scene.save_to_binary(compiled_path / f'{scene_filenames}/{scene_filenames}.ply')
# for now also save it as a json as it is useful for debugging purposes, can be removed later
#scene.save_to_json(compiled_path / f'{scene_filenames}/{scene_filenames}.json', assets_path)   

# Create the renderer
#lightmap_renderer = Renderer(width=32, height=32, scene=scene, atmosphere_color=atmosphere_color, lightmap_mode=True)

# Initialize the lightmapper
#lightmapper = Lightmapper(scene=scene, renderer=lightmap_renderer)

# Generate the lightmaps
#lightmap_path = compiled_path / f'{scene_filenames}/{scene_filenames}.hdr'
#lightmapper.generate_lightmap(lightmap_path=lightmap_path, iterations=1, patch_resolution=0.0625)
# Clean up
#lightmap_renderer.destroy()

renderer = Renderer(
    width=viewport_width, 
    height=viewport_height, 
    scene=scene,
    atmosphere_color=atmosphere_color, 
    lightmap_mode=False, 
)

renderer.run()
renderer.destroy()

FileNotFoundError: [Errno 2] No such file or directory: 'D:/data/Informatik/GamesEngineering/Bayernstein/assets/compiled/Milestone4/Milestone4.json'